In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-midterm-p3/train.csv
/kaggle/input/2021-ml-midterm-p3/test.csv
/kaggle/input/2021-ml-midterm-p3/sample_submit.csv


In [2]:
# train, test, submit data 불러오기
train_data = pd.read_csv('/kaggle/input/2021-ml-midterm-p3/train.csv')
test_data = pd.read_csv('/kaggle/input/2021-ml-midterm-p3/test.csv')
submit = pd.read_csv('/kaggle/input/2021-ml-midterm-p3/sample_submit.csv')

print(train_data.head())
print(test_data.head())

   index  0  1  2   3   4   5  6  7  8  ...  55  56  57  58  59  60  61  62  \
0      0  0  0  5  13   9   1  0  0  0  ...   0   0   0   6  13  10   0   0   
1      1  0  0  0  12  13   5  0  0  0  ...   0   0   0   0  11  16  10   0   
2      2  0  0  0   4  15  12  0  0  0  ...   0   0   0   0   3  11  16   9   
3      3  0  0  7  15  13   1  0  0  0  ...   0   0   0   7  13  13   9   0   
4      4  0  0  0   1  11   0  0  0  0  ...   0   0   0   0   2  16   4   0   

   63  label  
0   0      0  
1   0      1  
2   0      2  
3   0      3  
4   0      4  

[5 rows x 66 columns]
   index    0    1     2     3     4     5     6    7    8  ...    54   55  \
0     23  0.0  1.0   8.0  12.0  15.0  14.0   4.0  0.0  0.0  ...   0.0  0.0   
1     29  0.0  0.0   9.0  13.0   7.0   0.0   0.0  0.0  0.0  ...  16.0  2.0   
2     30  0.0  0.0  10.0  14.0  11.0   3.0   0.0  0.0  0.0  ...   0.0  0.0   
3     32  0.0  2.0  13.0  16.0  16.0  16.0  11.0  0.0  0.0  ...   0.0  0.0   
4     44  0.0  0.0   9

In [3]:
# index와 0이 학습하는데 필요 없는 인덱스라고 판단하여 버리고, train data의 label값은 y데이터이므로 드랍
# x, y, test데이터 나누기
x_data = train_data.drop(['index', 'label', '0'], axis = 1)
y_data = train_data['label']
test_data = test_data.drop(['index', '0'], axis = 1)

In [4]:
# 데이터를 보니까 정규화가 필요한 것 같다고 판단하여 scaler 사용
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
x_data_mn = scaler2.fit_transform(x_data)
test_data_mn = scaler2.transform(test_data)

In [5]:
# y_data를 판단해보니 범주형 데이터이므로 분류 방법 사용
y_data

0       0
1       1
2       2
3       3
4       4
       ..
1432    9
1433    0
1434    8
1435    9
1436    8
Name: label, Length: 1437, dtype: int64

In [6]:
# knn 분류 시행 - 하이퍼파라미터는 거리에 가중치를 부여하고,주변 7개의 관측치 사용
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(weights = 'distance', n_neighbors = 7)
knn.fit(x_data_mn, y_data)
print(knn.score(x_data_mn, y_data))

1.0


In [7]:
# 로지스틱 회귀 모델 사용
from sklearn.linear_model import LogisticRegression
lgt = LogisticRegression(solver = 'sag', max_iter = 500)
lgt.fit(x_data_mn, y_data)
print(lgt.score(x_data_mn, y_data))

0.9874739039665971


In [8]:
# 선형판별분석 모델 사용
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_data_mn, y_data)
print(lda.score(x_data_mn, y_data))

0.9693806541405706


In [9]:
# 각각 모델의 score을 보니 knn이 가장 뛰어나다고 판단하여 knn으로 제출
pred_knn = knn.predict(test_data_mn)
submit['label'] = pred_knn
submit.to_csv('submit_knn.csv', index = False)

In [10]:
# 실험결과
# y데이터의 값을 보고 분류라고 판단하였고, 분류 문제에 대해 풀기 위해 분류 기계학습 모델을 사용하여 예측점수를 비교해본 결과
# knn이 가장 뛰어냤음. 데이터 전처리 과정에서는 데이터의 분포를 보니까 0걊이 많았고, 이 때문에 minmax스케일러를 사용햐여 전처리 해줌
# 또한 0인덱스와 Index인덱스가 학습과정에서 별로 필요없다고 판단하여 드랍시킵